# <a id="index">Index</a>
0. [Setup](#setup)
1. [Dataset Classes](#datasetClasses)
2. [Model Classes](#modelClasses)
3. [Train Function](#trainFunction)
4. [Test Function](#testFunction)
5. [Checkpoint Classes](#checkpointClasses)
6. [Train](#train)

# <a id="setup">0. Setup</a>
[Back To Index](#index)

In [1]:
import torch
import numpy as np
from pathlib import Path
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, Dataset
import torchvision
import torch.nn as nn
from torchvision import transforms
import os
import glob
import os.path as osp
import matplotlib.pyplot as plt
%matplotlib inline


import torch.nn as nn            # containing various building blocks for your neural networks
import torch.optim as optim      # implementing various optimization algorithms
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface

# torchvision: popular datasets, model architectures, and common image transformations for computer vision.
import torchvision
# transforms: transformations useful for image processing
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

import glob
import os.path as osp
import numpy as np
from PIL import Image

In [2]:
#path=os.getcwd()+'/training_data_numpy/'
path='../project_data/'

In [16]:
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


# <a id="datasetClasses">1. Dataset Classes</a>
[Back To Index](#index)

In [25]:
class MammogramDataset(Dataset):
    def __init__(self):
        self.files_features=[]
        self.files_labels=[]
        self.transform=transforms.ToTensor()
        
        #Features
        
        feature_files=glob.glob(osp.join(path,'*count*.npz'))
        feature_files=sorted(feature_files)
        
        for f in feature_files:
                #num_imgs=int(f[f.index(' '):f.index('.')])
                num_imgs=int(f.split(" ")[1].split(".")[0])
                for i in range(num_imgs):
                    self.files_features.append(f[:f.index('.npz')]+'-'+str(i)+'.npz')
        
        self.len=len(self.files_features)
        
        
        #Labels
        
        labels_files=glob.glob(osp.join(path,'*class_arr.npz'))
        labels_files=sorted(labels_files)
        
        first_array=np.load(labels_files[0])
        training_labels=first_array['arr_0']
        
        for f in range(len(labels_files)-1):
            array= np.load(labels_files[f])
            training_labels=np.concatenate((training_labels,array['arr_0']),axis=0)
        
        self.training_labels=training_labels
        
    
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        file=self.files_features[index]
        file_name=file[:file.index('-')]+'.npz'
        
        image_number=int(file.split("-")[1].split(".npz")[0])
        array_feature= np.load(file_name)
        selected_feature=array_feature['arr_0'][:,:,image_number]
        
        label=self.training_labels[index]
        
        
        return self.transform(selected_feature), label
    
    def __len__(self):
        return self.len

In [26]:
mam=MammogramDataset()

In [27]:
trainset_loader = data_utils.DataLoader(mam, batch_size=16, shuffle = True)

In [41]:
testset_loader = data_utils.DataLoader(mam, batch_size=16, shuffle = True)

# <a id="modelClasses">2. Model Classes</a>
[Back To Index](#index)

In [28]:
# TODO how to make the model flexible t different input shapes
#AdaptiveAvgPooling??

In [29]:
# With Sequential

In [30]:
class NetSeq(nn.Module):
    def __init__(self):
        super(NetSeq, self).__init__()

        # conv layers: feature extractor
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        
        self.conv_layers_output_dim=[20,71,71]
        self.conv_layers_output_dim=np.prod(self.conv_layers_output_dim)
        
        self.num_classes=5
        
        # fc layers: classifier
        self.fc_layers = nn.Sequential(
            nn.Linear(self.conv_layers_output_dim, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, self.num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, self.conv_layers_output_dim)
        x = self.fc_layers(x)
        return F.log_softmax(x, dim=1)

model = NetSeq().to(device, dtype=torch.float)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# <a id="checkpointClasses">5. Checkpoint Classes</a>
[Back To Index](#index)

In [38]:
def save_checkpoint(checkpoint_path, model, optimizer):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    # - for an optimizer, contains info about the optimizer’s states and hyperparameters used.
    state = {
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

# <a id="trainFunction">3. Train Function</a>
[Back To Index](#index)

In [35]:
def train(epoch, num_iter=10, save_interval=500, log_interval=1):
    model.train()  # set training mode
    iteration = 0
    for ep in range(epoch):
        start = time()
        
        # for i in range(num_iter)
        #dataiter = iter(trainset_loader)
        #images, labels = dataiter.next()
        for batch_idx, (data, target) in enumerate(trainset_loader):

            # bring data to the computing device, e.g. GPU
            data, target = data.to(device, dtype=torch.float), target.to(device)

            # forward pass
            output = model(data)
            # compute loss: negative log-likelihood
            loss = F.nll_loss(output, target)
            
            # backward pass
            # clear the gradients of all tensors being optimized.
            optimizer.zero_grad()
            # accumulate (i.e. add) the gradients from this forward pass
            loss.backward()
            # performs a single optimization step (parameter update)
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1
            
            if iteration % save_interval == 0 and iteration > 0:
                save_checkpoint('mamo-%i.pth' % iteration, model, optimizer)
            iteration += 1
            
        end = time()
        print('{:.2f}s'.format(end-start))
        test() # evaluate at the end of epoch

# <a id="testFunction">4. Test Function</a>
[Back To Index](#index)

In [42]:
def test():
    model.eval()  # set evaluation model
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testset_loader:
            data, target = data.to(device, dtype=torch.float), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testset_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testset_loader.dataset),
        100. * correct / len(testset_loader.dataset)))

# <a id="trainLoop">6. Train</a>
[Back To Index](#index)

In [37]:
train(epoch=2, num_iter=10, save_internal=10, log_interval=10)

Train Epoch: 0 [0/55885 (0%)]	Loss: 0.540130
Train Epoch: 0 [16/55885 (0%)]	Loss: 0.363854
Train Epoch: 0 [32/55885 (0%)]	Loss: 0.328436
Train Epoch: 0 [48/55885 (0%)]	Loss: 0.545737
Train Epoch: 0 [64/55885 (0%)]	Loss: 0.818736
Train Epoch: 0 [80/55885 (0%)]	Loss: 1.427752
Train Epoch: 0 [96/55885 (0%)]	Loss: 0.438405
Train Epoch: 0 [112/55885 (0%)]	Loss: 1.022010
Train Epoch: 0 [128/55885 (0%)]	Loss: 1.426332
Train Epoch: 0 [144/55885 (0%)]	Loss: 0.639318
Train Epoch: 0 [160/55885 (0%)]	Loss: 0.833664
Train Epoch: 0 [176/55885 (0%)]	Loss: 0.722047
Train Epoch: 0 [192/55885 (0%)]	Loss: 0.925011
Train Epoch: 0 [208/55885 (0%)]	Loss: 1.190951
Train Epoch: 0 [224/55885 (0%)]	Loss: 0.855057
Train Epoch: 0 [240/55885 (0%)]	Loss: 0.856953
Train Epoch: 0 [256/55885 (0%)]	Loss: 1.968485
Train Epoch: 0 [272/55885 (0%)]	Loss: 0.477046
Train Epoch: 0 [288/55885 (1%)]	Loss: 0.997791
Train Epoch: 0 [304/55885 (1%)]	Loss: 1.038087
Train Epoch: 0 [320/55885 (1%)]	Loss: 0.429615
Train Epoch: 0 [336/5

KeyboardInterrupt: 

In [ ]:
test()

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
import datetime
datetime.datetime.now()

## Scratch

In [9]:
#INPUT
batch_input = torch.randn(16,1,299, 299)
print("batch_input shape", batch_input.shape)


#FIRST BLOCK
conv_layer_1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
output_conv_layer_1=conv_layer_1(batch_input)
print("output_conv_layer_1.shape", output_conv_layer_1.shape)

x = x.view(-1, 320)

batch_input shape torch.Size([16, 1, 299, 299])


torch.Size([16, 20, 71, 71])

In [13]:
np.prod([1,2,3])

6